In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML

from datasets import load_metric, load_from_disk, Dataset, DatasetDict

### 데이터 불러오기

In [4]:
cd ..

/opt/ml


In [5]:
datasets = load_from_disk("./data/train_dataset")

In [6]:
train = datasets['train']
val = datasets['validation']

In [7]:
train_df = pd.DataFrame(train)
val_df = pd.DataFrame(val)

In [9]:
train_answer = [''.join(train_df['answers'][i]['text']) for i in range(len(train_df))]
val_answer = [''.join(val_df['answers'][i]['text']) for i in range(len(val_df))]

train_context = train_df['context']
val_context = val_df['context']

train_question = train_df['question']
val_question = val_df['question']

** **TO DO !!** **
1. train, validation의 answer, context, question을  roberta의 tokenizer로 토크나이징
2. 토크나이징된 각 데이터를 Pororo와 Koanlan으로 NER 태깅

BUT, Koalan은 일일 API 사용량이 정해져있어 안될 수도 있다. -> APIException

# Pororo

In [10]:
from pororo import Pororo

ner = Pororo(task="ner", lang="ko")

In [11]:
from transformers import AutoModel, AutoTokenizer

# model = AutoModel.from_pretrained("klue/roberta-large")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

### 데이터 불러오기 & '##' 지우기

In [20]:
train_tokenized_ans_org = [tokenizer.tokenize(data) for data in train_answer]
train_tokenized_ctx_org = [tokenizer.tokenize(data) for data in train_context]
train_tokenized_qst_org = [tokenizer.tokenize(data) for data in train_question]

val_tokenized_ans_org = [tokenizer.tokenize(data) for data in val_answer]
val_tokenized_ctx_org = [tokenizer.tokenize(data) for data in val_context]
val_tokenized_qst_org = [tokenizer.tokenize(data) for data in val_question]

In [21]:
train_tokenized_ans_org

[['하원'],
 ['《', '경영', '##의', '실제', '》'],
 ['백성'],
 ['중국'],
 ['4', '##개'],
 ['드래곤'],
 ['형', '##양', '##태', '##수', '왕', '##식'],
 ['이탈리아', '##군'],
 ['큰', '##아들', '유'],
 ['왕', '##대', '##마을'],
 ['음', '##독', '##자', '##살'],
 ['출장', '잦', '##은', '건축가'],
 ['반신', '##화', '##적인', '인물', '##인', '우파'],
 ['1951', '##년'],
 ['예수'],
 ["'", '초일', '##기', "'"],
 ['1916', '##년'],
 ['레드', '##삭', '##스'],
 ['삼', '##판', '##동'],
 ['다산', '정약용'],
 ['대', '##나라'],
 ['10', '##달', '##러'],
 ['1945', '##년'],
 ['사립', '김', '##노', '##인', '고등학교'],
 ['체', '##벽', '##에', '잇', '##는', '감지', '##기'],
 ['근대', '##철', '##학'],
 ['215', '##k', '##m'],
 ['마', '##커트', '##리', '##로', '장식', '##을', '한', '트', '##롱', '##프', '뢰', '##유'],
 ['아르', '##투', '##로'],
 ['조립', '라인'],
 ['우', '##전', '##왕', '(', '[UNK]', '[UNK]', '王', ')', '식', '착', '##의', '##법'],
 ['친구'],
 ['2015', '##년'],
 ['케이스', '커', '##크'],
 ['직사', '##각형'],
 ['상인'],
 ['중국'],
 ['지상', '##요', '##원', '##들'],
 ['시조', '거인'],
 ["'", '발라', '##즈', '찬', '카이', '##이', "'"],
 ['1964', '##년', '12', '#

In [18]:
# 샾샾 지워 ~~
def remove_sharp(tokenized_data) :
    no_sharp = []
    for idx in range(len(tokenized_data)) :
        result = ''
        for item in tokenized_data[idx] :
            if '##' in item :
                result += item.replace('##', ' ')
            else :
                result += ' ' + item
        no_sharp.append(result.lstrip())
    return no_sharp


In [22]:
train_tokenized_ans = remove_sharp(train_tokenized_ans_org)
train_tokenized_ctx = remove_sharp(train_tokenized_ctx_org)
train_tokenized_qst = remove_sharp(train_tokenized_qst_org)

val_tokenized_ans = remove_sharp(val_tokenized_ans_org)
val_tokenized_ctx = remove_sharp(val_tokenized_ctx_org)
val_tokenized_qst = remove_sharp(val_tokenized_qst_org)

### NER 하기

In [28]:
# 토큰별로 NER ~~
def go_NER(tokenized_data) :
    NER_list = []
    for item in tokenized_data :
        result = ''
        tokens = item.split()
        for token in tokens :
            try :
                if token == ' ' :
                    pass
                else :
                    pred = ner(token)
                    result += " {}({}) ".format(pred[0][0], pred[0][1])
            except :
                result += token
        NER_list.append(result)
    return NER_list

In [30]:
train_NER_ans = go_NER(train_tokenized_ans)
train_NER_ctx = go_NER(train_tokenized_ctx)
train_NER_qst = go_NER(train_tokenized_qst)

In [31]:
val_NER_ans = go_NER(val_tokenized_ans)
val_NER_ctx = go_NER(val_tokenized_ctx)
val_NER_qst = go_NER(val_tokenized_qst)

In [88]:
train_NER_ctx[2]

' 강희(PERSON)  제(O)  는(O)  강화(O)  된(O)  황(PERSON)  권(PERSON)  으로(O)  거의(O)  황제(OCCUPATION)  중심(O)  의(O)  독단(O)  적으로(O)  나라(O)  를(O)  이끌(O)  어(O)  갔(O)  기에(O)  자칫(O)  전제(O)  독재(O)  의(O)  가능(O)  성이(O)  보일(O)  수도(O)  있(O)  었(O)  으나(O)  ,(O)  스스로(O)  황(PERSON)  권(PERSON)  을(O)  조절(O)  하고(O)  정치(O)  의(O)  일부(O)  는(O)  재상(CIVILIZATION)  들이(O)  나(O)  대신(O)  들(O)  과(O)  의논(O)  하(O)  였(O)  으며(O)  당시(O)  궁핍(O)  하(O)  게(O)  살(O)  고(O)  있(O)  는(O)  한족(CIVILIZATION)  들(O)  의(O)  사정(O)  을(O)  잘(O)  알고(O)  있(O)  던(O)  한족(CIVILIZATION)  대신(O)  들(O)  의(O)  의견(O)  을(O)  수용(O)  하여(O)  정책(O)  을(O)  실행(O)  하고(O)  선정(O)  을(O)  베풀(O)  었(O)  다(O)  .(O)  프랑스(COUNTRY)  의(O)  예수(PERSON)  회(O)  선교사(OCCUPATION)  부(O)  베(O)  는(O)  루이(PERSON)  14(QUANTITY)  세(QUANTITY)  에(O)  게(O)  다음(O)  과(O)  같이(O)  보고(O)  하(O)  였(O)  다(O)  .(O)  [(O)  n(O)  강(LOCATION)  희(PERSON)  제(O)  는(O)  세상(O)  에서(O)  가장(O)  부유(O)  한(O)  군주(OCCUPATION)  입니다(O)  .(O)  그럼에도(O)  황제(OCCUPATION)  인(O)  그(O)  의(O)  생활(O)  용품(O)  들(O)  은(MATERIAL)  

In [87]:
train_NER_qst[:20]

[' 대통령(CIVILIZATION)  을(O)  포함(O)  한(O)  미국(COUNTRY)  의(O)  행정부(ORGANIZATION)  견제(O)  권(PERSON)  을(O)  갖(O)  는(O)  국가(O)  기관(O)  은(MATERIAL)  ?(O) ',
 ' 현대(ORGANIZATION)  적(O)  인사(O)  조(O)  직(O)  관리(O)  의(O)  시발점(O)  이(O)  된(O)  책(O)  은(MATERIAL)  ?(O) ',
 ' 강희(PERSON)  제(O)  가(O)  171(QUANTITY)  7(QUANTITY)  년(DATE)  에(O)  쓴(O)  글(O)  은(MATERIAL)  누구(O)  를(O)  위해(O)  쓰여(O)  졌(O)  는(O)  가(O)  ?(O) ',
 ' 11(DATE)  ~(O)  12(QUANTITY)  세기(O)  에(O)  제작(O)  된(O)  본(O)  존(PERSON)  불(O)  은(MATERIAL)  보통(O)  어떤(O)  나라(O)  의(O)  특징(O)  이(O)  전파(O)  되(O)  었(O)  나(O)  요(O)  ?(O) ',
 ' 명문(O)  이(O)  적힌(O)  유물(O)  을(O)  구성(O)  하(O)  는(O)  그릇(O)  의(O)  총(O)  개수(O)  는(O)  ?(O) ',
 ' 카드(O)  모스(PERSON)  의(O)  부하(CIVILIZATION)  들이(O)  간(O)  곳(O)  에(O)  는(O)  무엇(O)  이(O)  있(O)  었(O)  는(O)  가(O)  ?(O) ',
 ' 관우(PERSON)  를(O)  불태(O)  워(O)  죽이(O)  려(O)  한(O)  사람(O)  누구(O)  인(O)  가(O)  ?(O) ',
 ' 참(O)  호(O)  속(O)  에(O)  무기(O)  와(O)  장비(O)  를(O)  버리(O)  고(O)  도주(O)  한(O)  집단(O)  은(MATERIAL)  ?(O) ',
 ' 제(O)  2(

### 토큰 수 = 태그 수 가 맞는지 확인

In [37]:
def check_tokens_tags(NER_list, token_list, raw_data) :
    for idx in range(len(raw_data)) :
        list1 = NER_list[idx].split()
        list2 = token_list[idx].split()
        if len(list1) != len(list2) :
            return idx, NER_list[idx], token_list[idx]   

In [38]:
check_tokens_tags(train_NER_ans, train_tokenized_ans, train)

(62, '〈 마(PERSON)  브(PERSON)  루(O)  공(O) ( 公(O) )〉', '〈 마 브 루 공 ( 公 ) 〉')

In [43]:
chk1 = check_tokens_tags(train_NER_ans, train_tokenized_ans, train)
chk2 = check_tokens_tags(train_NER_ctx, train_tokenized_ctx, train)
chk3 = check_tokens_tags(train_NER_qst, train_tokenized_qst, train)

chk4 = check_tokens_tags(val_NER_ans, val_tokenized_ans, val)
chk5 = check_tokens_tags(val_NER_ctx, val_tokenized_ctx, val)
chk6 = check_tokens_tags(val_NER_qst, val_tokenized_qst, val)

총 3개의 데이터의 토큰과 태그 수가 맞지 않음

In [74]:
# train_answer => 1개
print(chk1)
print(len(chk1[1].split()), len(chk1[2].split()))

(62, '〈 마(PERSON)  브(PERSON)  루(O)  공(O) ( 公(O) )〉', '〈 마 브 루 공 ( 公 ) 〉')
8 9


In [72]:
# train_context => 1개
# print(chk2)
print(len(chk2[1].split()), len(chk2[2].split()))

465 466


In [75]:
# validation_context => 1개
# print(chk5)
print(len(chk5[1].split()), len(chk5[2].split()))

733 735


In [92]:
# 그냥 대충 csv로 저장했다 ...
import csv

with open('Tokenized_train_val.csv', 'w', newline='') as f :
    writer = csv.writer(f)
    writer.writerow(train_NER_ans)
    writer.writerow(train_NER_ctx)
    writer.writerow(train_NER_qst)
    writer.writerow(val_NER_ans)
    writer.writerow(val_NER_ctx)
    writer.writerow(val_NER_qst)

# Koalan

In [76]:
from koalanlp.Util import initialize, finalize

# 꼬꼬마와 ETRI 분석기의 2.0.4 버전을 참조합니다.
initialize(java_options="-Xmx4g", KKMA="2.0.4", ETRI="2.0.4")

# finalize() 사용 종료 필수

[koalanlp.jip] [Checking] pom file /opt/ml/.m2/repository/kr/bydelta/koalanlp-kkma/2.0.4/koalanlp-kkma-2.0.4.pom
[koalanlp.jip] [Skipped] pom file not found at /opt/ml/.m2/repository/kr/bydelta/koalanlp-kkma/2.0.4/koalanlp-kkma-2.0.4.pom
[koalanlp.jip] [Checking] pom file /opt/ml/.ivy2/cache/kr/bydelta/koalanlp-kkma/2.0.4/koalanlp-kkma-2.0.4.pom
[koalanlp.jip] [Skipped] pom file not found at /opt/ml/.ivy2/cache/kr/bydelta/koalanlp-kkma/2.0.4/koalanlp-kkma-2.0.4.pom
[koalanlp.jip] [Checking] pom file https://oss.sonatype.org/content/repositories/public/kr/bydelta/koalanlp-kkma/2.0.4/koalanlp-kkma-2.0.4.pom
[koalanlp.jip] [Checking] pom file /opt/ml/.m2/repository/kr/bydelta/koalanlp-core/2.0.4/koalanlp-core-2.0.4.pom
[koalanlp.jip] [Skipped] pom file not found at /opt/ml/.m2/repository/kr/bydelta/koalanlp-core/2.0.4/koalanlp-core-2.0.4.pom
[koalanlp.jip] [Checking] pom file /opt/ml/.ivy2/cache/kr/bydelta/koalanlp-core/2.0.4/koalanlp-core-2.0.4.pom
[koalanlp.jip] [Skipped] pom file not f

In [77]:
from koalanlp import API
from koalanlp.proc import EntityRecognizer

API_KEY = 'bf78c594-8d95-4a08-91d3-1797338ba45f'

recognizer = EntityRecognizer(API.ETRI, etri_key=API_KEY)

In [78]:
parsed_1 = recognizer(train_tokenized_ans[:1500])

In [79]:
parsed_2 = recognizer(train_tokenized_ans[1500:])

In [83]:
parsed_1[1]

《 경영 의 실제 》

In [82]:
parsed_1[1].getEntities()

[AF(AFW_DOCUMENT; '경영 의 실제')]